# <span style='font-family: CMU Sans Serif, sans-serif;'> Hidden units  </span> 

This issue is unique to feedforward neural networks: how to choose the type of hidden unit to use in the hidden layers of the model.

**ReLU are generally acceptable choices**. The following gives intuition on how to select what hidden units, however predicting what will work best before fitting the model is usually impossible.

Some hidden units below are not differentiable at all points (e.g. ReLU $g(z) = \max\{ 0,z \}$). This is not considered an issue since we do not expect to arrive at the global minima of the loss function, but merely reduce it significantly. Additionally, non-differentiable hidden units are usually only non-differentiable at few points and thus have left and right derivatives. Software returns either the left or right derivative in case of an undefined derivative (the justification is that we rarely have fx. $g(0)$ in the real world, but rather $g(0 + \epsilon)$ which is rounded to $0$).

Unless indicated otherwise, hidden units accept a vector of inputs $\mathbf{x}$, computes an affine transformation $\mathbf{z} = \mathbf{W}^{\top}\mathbf{x} + \mathbf{b}$, and then applies an element-wise nonlinear function $g(\mathbf{z})$ where $g(\cdot )$ distinguishes the hidden units.

## <span style='font-family: CMU Sans Serif, sans-serif;'> Rectified linear units and their generalizations  </span> 

ReLU is given by $g(z) = \max\{ 0,z \}$. 

These are easy to optimize because they are similar to linear units. In half of its domain it is $0$ which makes the derivative large whenever the the unit is active. The second derivative is almost $0$ everywhere, and the derivative is $1$ everywhere that the unit is active, and zero otherwise. This means the gradient is far more useful for learning than it would be with activation function that introduce second-order effect.

These are typically used on top of an affine transformation
$$
\mathbf{h} = g(\mathbf{W}^{\top}\mathbf{x} + \mathbf{b}) \tag{1}
$$

A drawback of the ReLU is that they cannot learn via the gradient-based methods where the activation is zero. This is addressed by the following generalizations

Three generalizations are based on using a non-zero slope $\alpha_i$ when $z_i<0$: $h_i = g(\mathbf{z}, \mathbf{\alpha})_i = \max(0,z_i) + \alpha_i \min(0,z_i)$.

**Absolute value rectification** fixes $\alpha_i = -1$ to obtain $g(z)=\vert z \vert$. This is used for object recognition from images, where we seek features that are invariant under a polarity reversal of the input illumination. 

**Leaky ReLU** fixes $\alpha_i$ to a small value like $0.01$, whilst **parametric ReLU**, PReLU, treats $\alpha_i$ as a learnable parameter.

**Maxout units** generalize rectified linear units further. Instead of applying an element-wise $g(z)$, maxout units divide $\mathbf{z}$ into groups of $k$ values. Each maxout unit then outputs the maximum element of one of these groups
$$
g(\mathbf{z})_i = \max_{j \in \mathbb{G}^{(i)}} z_j
$$
where $\mathbb{G}^{(i)}$ is the set of indices into the inputs for groups $i$, $\{ (i-1)k+1, \ldots ,ik \}$. These can learn piecewise linear, convex functions with up to $k$ pieces. They can be viewed as *learning the activation function* itself rather then just a relation between units. With large enough $k$ a maxout unit can learn to approximate any convex function with arbitrary fidelity; this includes the ReLU, leaky ReLU, PReLU, absolute value rectification, or a totally different function. The maxout unit is parametrized differently compared to these type of layers (ReLU, leaky ReLU, etc.), so the learning dynamics are different even when the maxout learns to implement the same function of $\mathbf{x}$ as one of the other layer types.

Each maxout unit is parametrized by $k$ weight vectors instead of one, so maxout units typically need more regularization than rectified units. They can work well without regularization if the training set is large and the number of pieces per unit is kept low.

A few other benefits from using maxout units are (1) gaining statistical and computational advantages by requiring fewer parameters, (2) due to redundancy introduced by the multiple filters in maxout units, they are more resistant to **catastrophic forgetting** (when NNs forget how to perform tasks they were trained on before).

These units are based on the principle that models are easier to optimize if they exhibit (nearly) linear behavior. This is also true for recurrent neural networks (for different reasons).

## <span style='font-family: CMU Sans Serif, sans-serif;'> Logistic and hyperbolic tangent  </span> 

Before ReLU NNs used logistic sigmoids 
$$
g(z) = \sigma(z)
$$
or the hyperbolic tangent activation function
$$
g(z) = \tanh(z)
$$
, where $\tanh(z) = 2\sigma(2z) - 1$. 

Sigmoidal units saturate across most of their domain, which can make gradient based learning difficult, and their use as hidden units in feedforward NNs is thus discouraged. Their use a output units can be reasonable with proper choice of cost function that undoes the saturation (e.g. negative log-likelihood).

When a sigmoidal function **must** be used the $\tanh$ usually does better. $\tanh$ is more closely related to the identity function since $\tanh(0) = 0$ (where $\sigma(0)=\frac{1}{2}$) making training in $\hat{y} = \mathbf{w}^{\top} \tanh\left( \mathbf{U}^{\top}\tanh(\mathbf{V}^{\top}\mathbf{x}) \right)$ as long as the activations are kept small.

Sigmoidal activation functions are more commonly used in networks other than feedforward neural networks.

## <span style='font-family: CMU Sans Serif, sans-serif;'> Other hidden units  </span> 

These are less frequently used. Also unpublished differentiable activation functions can also perform well.

It is very common to test different activation function when developing different neural networks. Below are a few especially useful and distinctive ones.

One possibility is using the identity activation function as hidden output (no activation function at all). However, if all elements of a neural network are linear, then the entire neural network becomes linear, but it is acceptable for some layers to be purely linear.

Consider a NN layer with $n$ inputs and $p$ outputs, $\mathbf{h} = g(\mathbf{W}^{\top}\mathbf{x} + \mathbf{b})$. This may be replaced with two layers, with one layer using weight matrix $\mathbf{U}$ and the other using $\mathbf{V}$. If the first has no activation function, then we have essentially factored the weight matrix of the original layer on $\mathbf{W}$. The factored approach is to compute $\mathbf{h} = g(\mathbf{V}^{\top}\mathbf{U}^{\top}\mathbf{x} + \mathbf{b})$. If $\mathbf{U}$ produces $q$ outputs then $\mathbf{U}$ and $\mathbf{V}$ together contain $(n+p)q$ parameters, while $\mathbf{W}$ contains $np$ parameters. For small $q$ this can be a large saving in parameters. This comes at the cost of low rank linear transformations, but they may suffice for many applications. We can therefore reduce the amount of parameters with linear hidden units.

Softmax units can also be used as hidden units. They represent a probability distribution over a discrete variable with $k$ possible values, so they may be used as a switch. These kinds of hidden units are more common in complex architectures that explicitly learn to manipulate data. 

A few other common hidden unit types include:
- **Radial bias function** (RBF), unit: $h_i = \exp\left( - \frac{1}{\sigma^{2}_i} \left\| \mathbf{W}_{:,i}-\mathbf{x} \right\|^{2} \right)$. This function becomes more active as $\mathbf{x}$ approaches a template $\mathbf{W}_{:,i}$, but since it saturates to $0$ it can be difficult to optimize.
- **Softplus**: $g(a) = \zeta(a) = \log(1+e^{a})$. This is a smooth version of the rectifier. A comparison showed that the rectifier is better, and this activation function is generally discouraged. This is a confusing result, but it is empirically true.
- **Hard tanh**. THis is shaped like the $\tanh$ and the rectifier, but unlike the latter, it is bounded $g(a) = \max(-1, \min(1,a))$. 